# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

### Summary meduza

1. Датасет содержит 865847 строк, 16 столбцов, из которых пустых 10 столбцов
2. Период дат
3. Количество рубрик
4. Дубликаты
5. 




In [1]:
import numpy as np
import pandas as pd
import datetime as DT

## gazeta

Загрузим датасет gazeta. Одновременно сделаем преобразование даты из строки

In [2]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [4]:
data_gazeta = pd.read_csv('../gazeta.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"',
                      error_bad_lines=False)

b'Skipping line 437258: expected 17 fields, saw 20\n'
b'Skipping line 591054: expected 17 fields, saw 20\n'
b'Skipping line 780302: expected 17 fields, saw 20\n'
b'Skipping line 793371: expected 17 fields, saw 20\n'
b'Skipping line 838172: expected 17 fields, saw 20\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Проверим преобразование
data_gazeta.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors                object
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [6]:
data_gazeta.shape

(865847, 16)

In [7]:
# Так выглядет датасет gazeta
data_gazeta.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2008-11-21 15:19:14,https://www.gazeta.ru/news/business/2008/11/21...,-,Бизнес,NaN,Госдума сокращает срок действия ставки экспорт...,Госдума приняла сегодня в первом чтении и сраз...,-,-,-,-,-,-,-,-,-
1,2012-09-27 10:54:51,https://www.gazeta.ru/science/news/2012/09/27/...,-,Наука,NaN,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...,-,-,-,-,-,-,-,-,-
2,2015-12-01 15:28:45,https://www.gazeta.ru/army/news/7957079.shtml,-,Армия,NaN,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...,-,-,-,-,-,-,-,-,-
3,2013-04-18 16:42:06,https://www.gazeta.ru/travel/news/2013/04/18/n...,-,Новости,NaN,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...,-,-,-,-,-,-,-,-,-
4,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,-,Политика,NaN,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,-,-,-,-,-,-,-,-,-


In [8]:
# Выясним в каким столбцах есть пустые данные
data_gazeta.isnull().any()

date           False
url            False
edition        False
topics         False
authors         True
title          False
text           False
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [9]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_gazeta['authors'].isnull().sum())
data_gazeta[data_gazeta['authors'].isnull()]

Кол-во строк с значением NaN в столбце "authors" =  801505


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2008-11-21 15:19:14,https://www.gazeta.ru/news/business/2008/11/21...,-,Бизнес,NaN,Госдума сокращает срок действия ставки экспорт...,Госдума приняла сегодня в первом чтении и сраз...,-,-,-,-,-,-,-,-,-
1,2012-09-27 10:54:51,https://www.gazeta.ru/science/news/2012/09/27/...,-,Наука,NaN,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...,-,-,-,-,-,-,-,-,-
2,2015-12-01 15:28:45,https://www.gazeta.ru/army/news/7957079.shtml,-,Армия,NaN,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...,-,-,-,-,-,-,-,-,-
3,2013-04-18 16:42:06,https://www.gazeta.ru/travel/news/2013/04/18/n...,-,Новости,NaN,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...,-,-,-,-,-,-,-,-,-
4,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,-,Политика,NaN,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,-,-,-,-,-,-,-,-,-
5,2012-09-26 21:11:33,https://www.gazeta.ru/culture/news/2012/09/26/...,-,Культура,NaN,"Министерство культуры назвало фильмы, на котор...",Министерство культуры России определило перече...,-,-,-,-,-,-,-,-,-
6,2011-02-24 16:40:29,https://www.gazeta.ru/news/sport/2011/02/24/n_...,-,Спорт,NaN,Иванов возглавил совет ПБК ЦСКА,Заместитель председателяnправительства РФnСерг...,-,-,-,-,-,-,-,-,-
7,2012-09-27 10:32:37,https://www.gazeta.ru/culture/news/2012/09/27/...,-,Культура,NaN,Коллегия Минкультуры обсудила отказ от бессроч...,Отказ от бессрочных контактов с артистами и вв...,-,-,-,-,-,-,-,-,-
8,2012-09-27 10:36:58,https://www.gazeta.ru/culture/news/2012/09/27/...,-,Культура,NaN,Театр «Россия» на Пушкинской площади открылся ...,Мэр Москвы Сергей Собянин в среду официально о...,-,-,-,-,-,-,-,-,-
9,2012-09-27 11:49:45,https://www.gazeta.ru/culture/news/2012/09/27/...,-,Культура,NaN,Речь Брейвика в суде станет основой театрально...,Немецко-швейцарская театральная труппа намерен...,-,-,-,-,-,-,-,-,-


In [11]:
print('authors: ',data_gazeta['authors'].unique())

authors:  [nan 'Александра Кошкина'
 'Анастасия Берсенева,Максим Солопов,Константин Новиков,Евгений Сафронов'
 ... 'Алина Распоповой' 'Анатолий Караваев,Юрий Воронцов'
 'Евгения Петрова (Сочи)']


Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [10]:
print('edition: ',data_gazeta['edition'].unique())
print('reposts_fb: ', data_gazeta['reposts_fb'].unique())
print('reposts_vk: ',data_gazeta['reposts_vk'].unique())
print('reposts_ok: ',data_gazeta['reposts_ok'].unique())
print('reposts_twi: ',data_gazeta['reposts_twi'].unique())
print('reposts_lj: ',data_gazeta['reposts_lj'].unique())
print('reposts_tg: ',data_gazeta['reposts_tg'].unique())
print('likes: ',data_gazeta['likes'].unique())
print('views: ',data_gazeta['views'].unique())
print('comm_count: ',data_gazeta['comm_count'].unique())

edition:  ['-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


### Рубрики gazeta

In [11]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_gazeta.topics.unique().shape)
data_gazeta['topics'].unique()

Количество рубрик =  (12,)


array(['Бизнес', 'Наука', 'Армия', 'Новости', 'Политика', 'Культура',
       'Спорт', 'Общество', 'Стиль', 'Технологии', 'Авто', 'Мнения'],
      dtype=object)

Все рубрики, с количеством новостей в каждой группе

In [12]:
data_gazeta.groupby('topics')['topics'].count()

topics
Авто           46477
Армия          25522
Бизнес        105386
Культура       42290
Мнения           890
Наука          17258
Новости        13252
Общество      163389
Политика      100214
Спорт         318908
Стиль          12875
Технологии     19386
Name: topics, dtype: int64

In [13]:
data_gazeta.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2008-11-21 15:19:14,https://www.gazeta.ru/news/business/2008/11/21...,-,Бизнес,NaN,Госдума сокращает срок действия ставки экспорт...,Госдума приняла сегодня в первом чтении и сраз...,-,-,-,-,-,-,-,-,-
1,2012-09-27 10:54:51,https://www.gazeta.ru/science/news/2012/09/27/...,-,Наука,NaN,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...,-,-,-,-,-,-,-,-,-
2,2015-12-01 15:28:45,https://www.gazeta.ru/army/news/7957079.shtml,-,Армия,NaN,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...,-,-,-,-,-,-,-,-,-
3,2013-04-18 16:42:06,https://www.gazeta.ru/travel/news/2013/04/18/n...,-,Новости,NaN,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...,-,-,-,-,-,-,-,-,-
4,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,-,Политика,NaN,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,-,-,-,-,-,-,-,-,-


### Deep analysis

In [ ]:
# Определить период дат

In [ ]:
# сколько строк пустых?